<a href="https://colab.research.google.com/github/Nganle21/Big-Data-Dreamteam/blob/main/Ngan/PF_songfeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

cid = '6c8892a3331d4602830359f009d42765'
secret = '2b147051b73d415c891fccec0d139b15'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
#from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
from collections import Counter

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
name = ["Miley Cirus"]
result = sp.search(name) 
result['tracks']['items'][1]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/5YGY8feqx7naU7z4HrwZM6'},
  'href': 'https://api.spotify.com/v1/artists/5YGY8feqx7naU7z4HrwZM6',
  'id': '5YGY8feqx7naU7z4HrwZM6',
  'name': 'Miley Cyrus',
  'type': 'artist',
  'uri': 'spotify:artist:5YGY8feqx7naU7z4HrwZM6'}]

In [6]:
billboard['aSearch'] = billboard['artist'].apply(lambda x : " ".join(x.split(" ")[:2]))
billboard['searcher'] = billboard['song'] + " " + billboard['aSearch']
billboard.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board,aSearch,searcher
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3,Adele,Easy On Me Adele
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16,The Kid,Stay The Kid
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14,Lil Nas,Industry Baby Lil Nas
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19,Walker Hayes,Fancy Like Walker Hayes
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18,Ed Sheeran,Bad Habits Ed Sheeran


In [7]:
billboard = billboard[billboard["artist"]=="Ed Sheeran"]
billboard = billboard["searcher"].unique()
billboard.shape


(26,)

In [8]:
billboard = pd.DataFrame(data = billboard, columns = ["searcher"])

In [ ]:
def getTrackURI(searcher):
    result = sp.search(searcher)
    try:
        obj1 = result['tracks']['items'][0]['uri']
        
    except:
        obj1 = None
    return obj1

def getArtists(searcher):
    result = sp.search(searcher)
    try:
        obj2 = result['tracks']['items'][0]['artists']
        names = []
        for name in obj2:
            names.append(name['name'])
        obj2 = ",".join(names)      
    except:
        obj2 = ""
    return obj2



billboard['uri'] = billboard['searcher'].apply(getTrackURI)
billboard['artists'] = billboard['searcher'].apply(getArtists)

billboard = billboard.dropna()
billboard.head()

In [10]:
audio_features = []
for uri in billboard['uri'].values:
    audio_features.append(sp.audio_features(uri)[0])

In [11]:
audio_features = pd.DataFrame(audio_features)[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                             'acousticness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
audio_features['duration'] = audio_features['duration_ms']/60000
audio_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,liveness,valence,tempo,duration_ms,time_signature,duration
0,0.807,0.893,11,-3.745,0,0.0347,0.0451,0.3660,0.537,126.011,230747,4,3.845783
1,0.788,0.859,2,-2.724,1,0.0856,0.2810,0.0424,0.822,141.020,207853,4,3.464217
2,0.471,0.396,8,-6.654,1,0.0336,0.7700,0.0729,0.263,149.609,215507,4,3.591783
3,0.641,0.324,11,-5.851,1,0.0299,0.6980,0.3280,0.273,110.184,185487,4,3.091450
4,0.598,0.448,8,-6.312,1,0.0232,0.1620,0.1060,0.168,95.009,263400,3,4.390000


In [12]:
billDf = pd.concat([billboard, audio_features], axis=1)

In [ ]:
newOrderURI = "spotify:artist:1uNFoZAHBGtllmzznpCI3s"
results = sp.artist_albums(newOrderURI, album_type='album')
albums = results['items']
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])
albums_list = []

#find album id and put all together in one list
for album in albums:
    id = album['id']
    albums_list.append(id)
albums_list


In [116]:
#find all avalble tracks via list of album
tracks_list = []
for i, album in enumerate(albums_list):
        track = sp.album_tracks(album)
        tracks_list.append(track)
        


In [ ]:
tracks_uri = []
tracks_name = []
#for a in range(len(albums_list))
for list in tracks_list:
    for i in range(len(list)):
        track_uri = a['items'][i]['uri']
        track_name = a['items'][i]['name']
        tracks_uri.append(track_uri) 
        tracks_name.append(track_name) 
    
print(tracks_name)   



num_values = len(Counter(tracks_uri).keys())
print(num_values)

In [ ]:
tracks_uri = pd.DataFrame(data = tracks_uri, columns = ['uri'])


In [ ]:
audio_features = []
for uri in tracks_uri.values:
    audio_features.append(sp.audio_features(uri)[0])
audio_features = pd.DataFrame(audio_features)[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                             'acousticness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
audio_features['duration'] = audio_features['duration_ms']/60000
audio_features = pd.concat([tracks_uri,audio_features],axis=1)
audio_features.head()

,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,liveness,valence,tempo,duration_ms,time_signature,duration
0,spotify:track:6i4G85fdtDwK6UpQba337E,0.583,0.444,9,-8.601,1,0.0456,0.5930,0.3530,0.167,119.590,152796,4,2.546600
1,spotify:track:2t96tGXVqmNT9KbbIXyEP1,0.677,0.633,7,-7.831,1,0.0771,0.0314,0.1120,0.409,89.934,187239,4,3.120650
2,spotify:track:5RE1BlC2bVRL02T0wcLq2X,0.595,0.543,11,-8.149,1,0.0380,0.1270,0.0975,0.109,99.928,174406,4,2.906767
3,spotify:track:3Ic8V1CEjOY53OVpo2k1Ap,0.509,0.228,4,-8.680,1,0.0332,0.9200,0.1040,0.586,90.674,156468,4,2.607800
4,spotify:track:2HFHwTr1IBWheGoQtNNKtx,0.670,0.693,6,-8.340,1,0.3500,0.2000,0.0909,0.388,87.017,212093,4,3.534883


In [15]:
name = 'Justin Bieber'
a = sp.search(q='artist:' + name, type='artist')
a['artists']['items'][0]['id']

'1uNFoZAHBGtllmzznpCI3s'

In [16]:
artist_name= 'Justin Bieber'
artistID = (sp.search(q='artist:' + name, type='artist'))['artists']['items'][0]['id']